<a href="https://colab.research.google.com/github/dayanebravo/DSBD/blob/main/teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

#Parâmetros
letra='A'
numero='I'
dia=14
mes=2
ano=2022
modalidade = 'DISTÂNCIA|USA'
#listaDisciplinas=''
listaDisciplinas = 'Equações Diferenciais|Transformadas|Matemática Aplicada|Modelagem Estatística|Exploratória|Discreta|'
#listaDisciplinas = 'Pré-Cálculo'

#Linhas opcionais para visualizar os dados completos
pd.set_option("max_rows", None)
pd.set_option("max_colwidth", None)

#Carregando dados e limpando
df = pd.DataFrame(pd.read_html('dados.xls',encoding='utf-8')[0])
df.replace('\xa0', ' ',regex=True, inplace=True)
df = df.loc[df['Curso'].str.contains(modalidade,regex=True)]
df = df.loc[df['Oferta']!=0]
df = df.rename({'Data\xa0Integração\xa0Univirtus': 'Data'}, axis=1)
df['Data'] = pd.to_datetime(df['Data'],format='%d/%m/%y %H:%M')



#Descobre a primeira oferta do período
limiteOferta= 999999999999
for indice,linha in df.iterrows():
   if linha['Data'] > pd.to_datetime(f'{ano}-{mes}-{dia}'):
     if limiteOferta > linha['Oferta'] :
        limiteOferta= linha['Oferta']

#Buscando pela Fase adequada
fase=f'{letra} Fase {numero} |UTA {letra} 2021 - Fase {numero}$'
df=df.loc[df['Módulo'].str.contains(fase, flags=re.I,regex=True)]

#Limpando o campo com o nome do curso
df['Curso'] = df['Curso'].str.replace(' -.*', '',regex=True)
df['Curso'] = df['Curso'].str.replace('TECNOLOGIA EM', '',regex=True)
df['Curso'] = df['Curso'].str.replace('BACHARELADO EM', '',regex=True)
df['Curso'] = df['Curso'].str.replace('CST', '',regex=True)

#Cópia dos dados originais e filtro por disciplinas
if listaDisciplinas: df =df.append({'Sala':listaDisciplinas}, ignore_index=True)
original = df.copy()
df['Sala'] = df['Sala'].str.lower()
df['Curso'] = df['Curso'].str.lower()
df['Sala'] = df['Sala'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df['Curso'] = df['Curso'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


#Casos especiais
df['Sala'] = df['Sala'].str.replace('programacao paralela', 'computacao paralela')
df['Sala'] = df['Sala'].str.replace('optica', 'otica')
df['Sala'] = df['Sala'].str.replace('distribuicao de energia eletrica', 'distribuicao de energia')
df['Sala'] = df['Sala'].str.replace('interface humano', 'interacao humano')
df['Sala'] = df['Sala'].str.replace('informacoes gerenciais', 'informacao gerencial')
df['Sala'] = df['Sala'].str.replace('difererencial', 'diferencial')
df['Sala'] = df['Sala'].str.replace('etica empresarial', 'etica')
df['Sala'] = df['Sala'].str.replace('comunicacoes', 'comunicacao')
df['Sala'] = df['Sala'].str.replace('producoes', 'producao')
df['Sala'] = df['Sala'].str.replace('internet das coisas', 'iot')
for indice,linha in df.iterrows():
   if listaDisciplinas and indice+1 == df.shape[0]: break
   if linha['Sala'].find("raciais") != -1:
     df.at[indice,'Sala'] ='educacao etnico racial'
   if linha['Sala'].find("extensionista") != -1 or linha['Sala'].find("trabalho de conclusao") != -1 :
     df.at[indice,'Sala'] = df.at[indice,'Sala'] + " - " + df.at[indice,'Curso']
     original.at[indice,'Sala'] = original.at[indice,'Sala'] + " - " + original.at[indice,'Curso']

#simplicando nome das salas
df.replace('\(.*\)', '',regex=True, inplace=True)
df.replace('s', '',regex=True, inplace=True)
df.replace(' da | do ', ' de ',regex=True, inplace=True)
df.replace(' ', '',regex=True, inplace=True)


#Filtro por disciplinas
if listaDisciplinas:
  listaDisciplinas = df['Sala'].iloc[-1]
  df.drop(df.tail(1).index,inplace=True)
  df = df.loc[df['Sala'].str.contains(listaDisciplinas,regex=True)]


df['Sala'] = df['Sala'].str.replace('\W', '')


#agrupando e listando
grupos = df.groupby('Sala')['Oferta'].apply(list)
grupos = grupos.reset_index(name = 'Ofertas')
for indice,linha in grupos.iterrows():
  linha['Sala'] = original.loc[original['Oferta'] == linha['Ofertas'][0]]['Sala'].iloc[0]
  linha['Ofertas'].sort()

#Transformando a lista de ofertas em colunas, colorizando e exportando para excel
serieOfertas=grupos.Ofertas.apply(pd.Series).merge(grupos, right_index = True, left_index = True)
cols = list(serieOfertas.columns.values)
serieOfertas= serieOfertas[[cols[-2]] + cols[0:-2]]
serieOfertas=serieOfertas.style.apply(lambda x : ['background-color:yellow' if type(v) !=str and v >= limiteOferta else '' for v in x])
serieOfertas.to_excel('resultado.xlsx', engine='openpyxl',index=False)   

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: FutureWarning: The default value of regex will change from True to False in a future version.
